In [16]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.misc as misc
%matplotlib inline

In [80]:
#Reading in the data
data = np.load('MC_50_5_4.npz.npy')
trueCls = np.load('MC_50_5_4_reference_classes.npy')

In [81]:
data.shape

(50, 5, 4)

In [82]:
#Inferring the number of classes and data length
numData = data.shape[0]
numExperts = data.shape[1]
numClasses = data.shape[2]

In [100]:
#Initialization
alpha = [1, 1, 1, 1]
z = np.random.dirichlet(alpha, size=numData)
observed_classes = np.argmax(z, axis=1)
pi = np.random.dirichlet(alpha, size=1)[0]
lam = 10
lambda_mat = lam*np.eye( numClasses) + np.ones((numClasses, numClasses))
ConfMatList = []
for exp_index in range(numExperts):
    confMat = []
    for cls_idx in range(numClasses):
        confMat.append(np.random.dirichlet(lambda_mat[cls_idx,:],1)[0])
    ConfMatList.append(np.array(confMat))
confMatArray = np.array(ConfMatList)

In [101]:
confMatArray[:,1,:]

array([[ 0.00622724,  0.92067626,  0.05015753,  0.02293897],
       [ 0.03432603,  0.52166703,  0.4272846 ,  0.01672234],
       [ 0.06379113,  0.82654313,  0.03994267,  0.06972308],
       [ 0.05456899,  0.82029447,  0.03817707,  0.08695948],
       [ 0.04002401,  0.88567101,  0.0698409 ,  0.00446408]])

In [102]:
def compute_LL(data, confMatArray, z, pi, obs_classes):
    log_like = 0
    numData = data.shape[0]
    numExperts = data.shape[1]
    numClasses = data.shape[2]
    for data_idx in range(numData):
        data_val = data[data_idx]
        z_val = z[data_idx]
        obs_val = obs_classes[data_idx]
        for cls_idx in range(numClasses):
            log_like += z_val[cls_idx]*(np.log(pi[cls_idx]) + \
                    np.sum(np.multiply(np.log(confMatArray[:, obs_val, cls_idx]), \
                                data_val[:, cls_idx])))
    return log_like

In [103]:
def E_step(data, confMatArray, pi, obs_classes):
    numData = data.shape[0]
    numExperts = data.shape[1]
    numClasses = data.shape[2]
    
    posterior = np.zeros(z.shape)
    for data_idx in range(numData):
        data_val = data[data_idx]
        obs_val = obs_classes[data_idx]
        for cls_idx in range(numClasses):
            posterior[data_idx, cls_idx] += np.log(pi[cls_idx]) + \
            np.sum(np.multiply(np.log(confMatArray[:, obs_val, cls_idx]),\
                                data_val[:, cls_idx]))
    
    log_denum_sum = misc.logsumexp(posterior, axis=1)
    norm_posterior = np.exp(posterior - log_denum_sum[:, np.newaxis])
    return norm_posterior

In [104]:
def M_Step(data, confMatArray, z, pi, obs_classes):
    numData = data.shape[0]
    numExperts = data.shape[1]
    numClasses = data.shape[2]
    
    #computing pi update
    newPi = np.mean(z, axis=0)
    
    #computing Confusion Matrix updates
    newConfMatArray = np.array([0.1*np.ones((numClasses, numClasses)) for idx in range(numExperts)])
    for e_idx in range(numExperts):
        for cls_idx in range(numClasses):
            dat = data[observed_classes == cls_idx]
            z_dat = z[observed_classes == cls_idx]
            for row_cls_idx in range(numClasses):
                newConfMatArray[e_idx, cls_idx, row_cls_idx] = \
                np.sum(np.multiply(dat[:,e_idx,row_cls_idx], z_dat[:, row_cls_idx]))
    newConfMatArray = newConfMatArray/newConfMatArray.sum(axis=(2))[:,:,np.newaxis]
    return newConfMatArray, newPi

In [105]:
compute_LL(data, confMatArray, z, pi, observed_classes)

-171.04412885564813

In [106]:
for i in range(100):
    z = E_step(data, confMatArray, pi, observed_classes)
    ConfMatArray, pi = M_Step(data, confMatArray, z, pi, observed_classes)
    print "Likelihood in iteration %f is %f", i+1, compute_LL(data, confMatArray, z, pi, observed_classes)

Likelihood in iteration %f is %f 1 -72.3142767173
Likelihood in iteration %f is %f 2 -74.0777811798
Likelihood in iteration %f is %f 3 -74.6270279028
Likelihood in iteration %f is %f 4 -74.7437782776
Likelihood in iteration %f is %f 5 -74.7359700486
Likelihood in iteration %f is %f 6 -74.70361423
Likelihood in iteration %f is %f 7 -74.6752881357
Likelihood in iteration %f is %f 8 -74.6565410229
Likelihood in iteration %f is %f 9 -74.6462821659
Likelihood in iteration %f is %f 10 -74.6421102399
Likelihood in iteration %f is %f 11 -74.641872465
Likelihood in iteration %f is %f 12 -74.6439820342
Likelihood in iteration %f is %f 13 -74.6473663643
Likelihood in iteration %f is %f 14 -74.6513373573
Likelihood in iteration %f is %f 15 -74.6554716587
Likelihood in iteration %f is %f 16 -74.6595202317
Likelihood in iteration %f is %f 17 -74.6633454855
Likelihood in iteration %f is %f 18 -74.6668793919
Likelihood in iteration %f is %f 19 -74.6700962877
Likelihood in iteration %f is %f 20 -74.672

In [107]:
np.argmax(z, axis=1)

array([0, 0, 0, 0, 1, 0, 1, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 3, 1, 3, 0, 3, 0, 0, 0, 1, 0, 1,
       0, 1, 3, 1])

In [108]:
trueCls

array([3, 1, 1, 2, 2, 2, 3, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 0, 0, 3, 2, 0, 2,
       0, 1, 1, 3, 3, 2, 2, 3, 0, 0, 3, 1, 0, 0, 0, 2, 0, 2, 3, 1, 3, 2, 2,
       3, 2, 2, 0])